In [1]:
import quantstats as qs
import yfinance as yf
import pandas as pd
import pandas_ta as ta
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display
from matplotlib.colors import LinearSegmentedColormap
from quantstats.stats import *
from quantstats._plotting.wrappers import distribution
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import df2img
from datetime import date

# my imports
import sys
sys.path.insert(1, '../scripts')
import utils as utl
import html_evaluation_portfolio as hep
import volatility_estimator as ve

In [36]:
#set overall parameters (list opf securities and window of days on which you compute the volatility on)

list_securities = ['SSO', 'UBT', 'UGL'] #  ['UPRO', 'TMF', 'IAU']  # ['UPRO', 'TMF', 'TYD']  # ['VTI', 'VGLT',  'IAU'] #['SPY', 'TLT', 'GLD'] #'VGLT', 'IAU']#['SSO', 'UBT', 'UGL'] # #'TYD'] #, 'DIG', 'UST'] #['SPY', 'TLT', 'GLD'] 'UTSL'] #DIG
window = 30 #days
title = 'x3 leverage'
estimator = 'stdv'
#start_date = pd.to_datetime('2007/01/03')
end_date = date.today() - pd.DateOffset(days=180)
# end_date = start_date + pd.DateOffset(days=1814) 
start_date = pd.to_datetime(end_date) - pd.DateOffset(days=1824) 

rebalance = 30 #days

if estimator not in ['stdv', 'parkinson', 'satchell']:
    raise ValueError('estimator has to be one of : stdv, parkison, satchell')

In [65]:
#main

data = hep.create_dataframe(list_securities) #I can import this dataframe but it is preatty fast.
#data['IAU'] = data['IAU'].replace([data['IAU'].max()], 34.599998)
#data['IAU'] =3* data['IAU']

if estimator == 'stdv':
    unique_name = 'stdv'
    for i in list_securities:
        unique_name += '_' + i
        
    df_strd_dev = ve.dataframe_strd_dev(data, window)
    df_strd_dev = ve.index_df(df_strd_dev, data, window)
    data_perc = hep.return_df(data)
    inv_volat_data = hep.df_inverse_volatility(data_perc, window)
    df_weighted = hep.df_wheighted(inv_volat_data)
    
if estimator == 'parkinson':
    unique_name = 'parkinson'
    for i in list_securities:
        unique_name += '_' + i

    data_h_l = ve.create_dataframe_high_low(list_securities)   
    #data_h_l = data_h_l[1135: 1135 + 1834]
    data_h_l = data_h_l.loc[start_date: end_date]
    df_volat = ve.compute_park_volatility(data_h_l, list_securities, window)
    data.dropna(inplace = True)
    #data = data[1135: 1135 + 1834]
    data = data[start_date: end_date]
    df_volat = ve.index_df(df_volat, data, window)
    df_inv_volat = df_volat.applymap(lambda x: 1/x)
    df_weighted = hep.df_wheighted(df_inv_volat)

if estimator == 'satchell':
    unique_name = 'satchell'
    for i in list_securities:
        unique_name += '_' + i
        
    data_ohlc = ve.create_dataframe_ohlc(list_securities)
    #data_holc = data_holc[1135: 1135 + 1834]
    data_ohlc = data_ohlc[start_date: end_date]
    df_satchell_volat = ve.compute_satchell_volatility(data_ohlc, list_securities, window)
    data.dropna(inplace = True)
    #data = data[1135: 1135 + 1834]
    data = data[start_date: end_date]
    df_satchell_volat = ve.index_df(df_satchell_volat, data, window-1)
    df_satchell_inv_volat = df_satchell_volat.applymap(lambda x: 1/x)
    df_weighted = hep.df_wheighted(df_satchell_inv_volat)
    
data_r = data.div(data.shift(1))-1
#data_r['IAU'] = 3*data_r['IAU']
data_earnings = hep.df_earnings(df_weighted, data_r)
data_earnings.dropna(inplace = True)
    
data_earnings = data_earnings[start_date: end_date]

In [96]:
df_weighted.dropna(inplace = True)
quantity_to_balance = pd.DataFrame(columns = df_weighted.columns)
for index in range(len(df_weighted)//rebalance):
    transition_array = []
    for sec in df_weighted.columns:
        df_weighted[sec].iloc[index*rebalance:(index+1)*rebalance] = df_weighted[sec][index*rebalance]
        if index > 0: 
            transition_array.append(df_weighted[sec].iloc[index*rebalance+1]-df_weighted[sec].iloc[index*rebalance-1])
    if len(transition_array) == 3:
        df_to_append = pd.DataFrame([transition_array], columns = df_weighted.columns, index = [df_weighted.index.tolist()[index*rebalance]])
        quantity_to_balance = pd.concat([quantity_to_balance, df_to_append])

In [97]:
df_weighted.index.tolist()[30]

Timestamp('2010-05-05 00:00:00')

In [98]:
pd.DataFrame([[1,2,3]], columns=['SSO', 'UBT', 'UGL'], index = [3])

#percentage of capital to sell or buy
#quantity_to_balance.index.tolist()

,SSO,UBT,UGL
3,1,2,3


In [123]:
capital = 10000
quantity_to_balance = quantity_to_balance * capital
number_of_sec_exchange = pd.DataFrame()

# iterate over the index and the row 
for index, row in quantity_to_balance.iterrows():
    number_of_sec_exchange_array = []
    # repete the passages for each security
    for sec in quantity_to_balance.columns:
        # append to the array the money I have to move over the price that day 
        # To get the numbers of trades necessary that day for the specific security
        number_of_sec_exchange_array.append(row[sec]/data.loc[index][sec])
    if len(number_of_sec_exchange_array) == len(quantity_to_balance.columns): # when it is done for every security, append th eresult to the dataframe
        #create dataframe to concatenate
        df_to_append = pd.DataFrame([number_of_sec_exchange_array], columns = quantity_to_balance.columns, index = [index])
        # concatenate the existence dataframe with the enw one
        number_of_sec_exchange = pd.concat([number_of_sec_exchange, df_to_append])    

In [122]:
number_of_sec_exchange

,SSO,UBT,UGL
2010-05-05,-175.771682,-8.766530,20.201393
2010-06-17,-156.952815,0.477503,12.364799
2010-07-30,77.570609,30.488987,-19.023922
2010-09-13,-101.492276,-102.464856,47.707551
2010-10-25,315.180460,57.854311,-44.022553
...,...,...,...
2021-10-12,-9.642431,-2.356702,12.717674
2021-11-23,20.935058,-13.659141,-13.500445
2022-01-06,-33.559995,8.065346,34.526556
2022-02-18,-0.253094,11.847607,-8.584374


In [117]:
data

,SSO,UBT,UGL
Date,,,
2007-01-03,9.372324,NaN,NaN
2007-01-04,9.420424,NaN,NaN
2007-01-05,9.271766,NaN,NaN
2007-01-08,9.349375,NaN,NaN
2007-01-09,9.325322,NaN,NaN
...,...,...,...
2022-06-03,53.259998,33.389999,59.500000
2022-06-06,53.560001,32.209999,58.930000
2022-06-07,54.570000,32.869999,59.700001
